Why Multi-Agent systems are required instead of one agent doing everything?
    1. the longer and larger the task we give to our agent, the more it's performance may start to suffer.

Let's say you need a quality market research. This workflow is quite long and may require different specialists skills at different times, like proposing an outline, researching on the web, validating your sources, writing the report, and then finally editing.

If you tried to hand this all off to one agent, it would have too many tools to choose from. Their context window would begin to overflow with information, and the quality of their output would suffer as a result.

A multi-agent system could break doiwn this complex application into multiple specialized agents that work together to solve the problem, rather than relying on a singular agent to handle every step.

We are going to build a supervisor sub-agent model, which uses a singular orchestrating agent delegating tasks to sub-agents.

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## Creating subagents

In [2]:
from langchain.tools import tool

@tool
def square_root(x: float) -> float:
    """Calculate the square root of a number"""
    return x ** 0.5

@tool
def square(x: float) -> float:
    """Calculate the square of a number"""
    return x ** 2

In [3]:
from langchain.agents import create_agent

# create subagents

subagent_1 = create_agent(
    model='gpt-4.1-nano',
    tools=[square_root]
)

subagent_2 = create_agent(
    model='gpt-4o-mini',
    tools=[square]
)

## Calling subagents

In [4]:
from langchain.messages import HumanMessage

@tool
def call_subagent_1(x: float) -> float:
    """Call subagent 1 in order to calculate the square root of a number"""
    response = subagent_1.invoke({"messages": [HumanMessage(content=f"Calculate the square root of {x}")]})
    return response["messages"][-1].content

@tool
def call_subagent_2(x: float) -> float:
    """Call subagent 2 in order to calculate the square of a number"""
    response = subagent_2.invoke({"messages": [HumanMessage(content=f"Calculate the square of {x}")]})
    return response["messages"][-1].content

## Creating the main agent

main_agent = create_agent(
    model='gpt-5-nano',
    tools=[call_subagent_1, call_subagent_2],
    system_prompt="You are a helpful assistant who can call subagents to calculate the square root or square of a number.")

## Test

In [5]:
question = "What is the square root of 456?"

response = main_agent.invoke({"messages": [HumanMessage(content=question)]})

In [6]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='What is the square root of 456?', additional_kwargs={}, response_metadata={}, id='7c6c4b55-335b-4b2e-b697-3c232610ada9'),
              AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_k5h2jiP7QdXRssSFisir8g0d', 'function': {'arguments': '{"x":456}', 'name': 'call_subagent_1'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 986, 'prompt_tokens': 202, 'total_tokens': 1188, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 960, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CwmKxPuOOXBfmLylTv863IuU2kF2f', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--b8cdae37-ad46-4e4a-9bd0-454f4c94444d-0', tool_calls=[{'name': 'call_subagent_1', 'args': {'x': 456}, 'id': '